# T5 Baseline

The initial exploration will use T5-small as the pre-training model along with ICSI dataset. When the model is ready, we will expand the dataset and also validation set for other hyperparameter tuning.

1. Library Loading  
2. Dataset Loading
3.   Dataset Transformation
4.   Training and Test Splitting
5.   Fine Tuning
6.   Checkpoint saving
7.   Evaluation



## Library Loading

In [1]:
!pip install 'transformers==3.3.1'
!pip install wandb -q
#!pip install datasets
!pip install nlp
#!pip install rouge_score
!pip install rouge
#!curl -q https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
#!python pytorch-xla-env-setup.py --apt-packages libomp5 libopenblas-dev
#!pip install sentencepiece

     |████████████████████████████████| 1.1MB 9.8MB/s 
     |████████████████████████████████| 3.0MB 23.9MB/s 
  Found existing installation: tokenizers 0.9.4
    Uninstalling tokenizers-0.9.4:
      Successfully uninstalled tokenizers-0.9.4
  Found existing installation: transformers 4.0.0
    Uninstalling transformers-4.0.0:
      Successfully uninstalled transformers-4.0.0


In [2]:
!pip list | grep transformers

transformers                  3.3.1          


In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import time

# Importing the T5 modules from huggingface/transformers
# T5ForConditionalGeneration is specific for sequence-to-sequence
from transformers import T5Tokenizer, T5ForConditionalGeneration

#from nlp import load_metric
import nlp
from rouge import Rouge
#import sentencepiece as spm
import wandb

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [4]:
# Checking out the GPU we have access to. This is output is from the google colab version. 
!nvidia-smi

Thu Dec  3 13:37:07 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0    29W / 250W |     10MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
# # Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

## Data Loading

Loaded from GDrive the transformed dataset.

This portion is using the dataset from extractive summary to abstractive summary

In [6]:
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
torch.backends.cudnn.deterministic = True

train_size = 0.8

In [7]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

#/content/drive/My Drive/W266/data/ICSI_extrac_abstrac_512token.csv

Mounted at /content/drive


## Dataset Transformation

Tokenize the input and also perform the attention masking to make sure everything can be done in tensors. 

Tunable Hyprparam:

*   MAX_LEN
*   SUMMARY_LEN
* TRAIN_BATCH_SIZE
* DEV_BATCH_SIZE
* TEST_BATCH_SIZE


In [95]:
# most code from https://colab.research.google.com/drive/1ypT7oCjtBOTSMJv7J5_1vO7hDYSD_-oU?authuser=2#scrollTo=932p8NhxeNw4

class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, source_len, summ_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = summ_len
        self.abstractive = self.data.abstractive
        self.extractive = self.data.extractive

    def __len__(self):
        return len(self.abstractive)

    def __getitem__(self, index):
        extractive = str(self.extractive[index])
        extractive = ' '.join(extractive.split())

        abstractive = str(self.abstractive[index])
        abstractive = ' '.join(abstractive.split())

        source = self.tokenizer.batch_encode_plus([extractive], max_length= self.source_len, truncation=True, pad_to_max_length=True,return_tensors='pt')
        target = self.tokenizer.batch_encode_plus([abstractive], max_length= self.summ_len, truncation=True, pad_to_max_length=True,return_tensors='pt')
        source_ids = source['input_ids'].squeeze()
        source_mask = source['attention_mask'].squeeze()
        target_ids = target['input_ids'].squeeze()
        target_mask = target['attention_mask'].squeeze()

        return {
            'source_ids': source_ids.to(dtype=torch.long), 
            'source_mask': source_mask.to(dtype=torch.long), 
            'target_ids': target_ids.to(dtype=torch.long),
            'target_ids_y': target_ids.to(dtype=torch.long)
        }

## Fine Tuning

Here we directly use the pre-trained model t5-small and will save checkpoint every 500 steps. 

Tunable Parameter:
* T5ForConditionalGeneration or T5
* epoch - train, dev, test
* optimizer - LEARNING_RATE, Adam
* output: num_beams, length_penalty,early_stopping




### Training & Validation Functions

The training part uses the t5-small pretrained model, didn't make any change to the model layer structures, and fine tune the parameters based on the dataset we have.

In [34]:
def compute_rouge_scores(cand_list, ref_list):
    """
    :param cand_list: list of candidate summaries
    :param ref_list: list of reference summaries
    :return: rouge scores
    """
    rouge = Rouge()
    rouge_1_f_score = 0.
    rouge_2_f_score = 0.
    rouge_L_f_score = 0.

    rouge_1_r_score = 0.
    rouge_2_r_score = 0.
    rouge_L_r_score = 0.

    rouge_1_p_score = 0.
    rouge_2_p_score = 0.
    rouge_L_p_score = 0.

    doc_count = len(cand_list)

    for cand, ref in zip(cand_list, ref_list):
        rouge_scores = rouge.get_scores(cand, ref)[0]
        rouge_1_f_score += rouge_scores['rouge-1']['f']
        rouge_2_f_score += rouge_scores['rouge-2']['f']
        rouge_L_f_score += rouge_scores['rouge-l']['f']

        rouge_1_r_score += rouge_scores['rouge-1']['r']
        rouge_2_r_score += rouge_scores['rouge-2']['r']
        rouge_L_r_score += rouge_scores['rouge-l']['r']

        rouge_1_p_score += rouge_scores['rouge-1']['p']
        rouge_2_p_score += rouge_scores['rouge-2']['p']
        rouge_L_p_score += rouge_scores['rouge-l']['p']
    rouge_1_f_score = rouge_1_f_score / doc_count
    rouge_2_f_score = rouge_2_f_score / doc_count
    rouge_L_f_score = rouge_L_f_score / doc_count

    results_dict = {}
    results_dict['rouge_1_f_score'] = rouge_1_f_score
    results_dict['rouge_2_f_score'] = rouge_2_f_score
    results_dict['rouge_l_f_score'] = rouge_L_f_score

    return results_dict

# Testing

In [125]:
test_dataset_icsi = pd.read_csv('/content/drive/My Drive/W266/data/512_tokens/ICSI_512_test.csv',encoding='latin-1')
test_dataset_ami = pd.read_csv('/content/drive/My Drive/W266/data/512_tokens/AMI_512_test.csv',encoding='latin-1')

test_dataset_icsi = test_dataset_icsi.dropna(subset=['abstractive'])
test_dataset_icsi = test_dataset_icsi.reset_index(drop=True)

test_dataset_ami = test_dataset_ami.dropna(subset=['abstractive'])
test_dataset_ami = test_dataset_ami.reset_index(drop=True)

# use the pre-defined "summarize" for abstractive summary
test_dataset_icsi.extractive = 'summarize: ' + test_dataset_icsi.extractive
test_dataset_ami.extractive = 'summarize: ' + test_dataset_ami.extractive
print(len(test_dataset_icsi))
print(len(test_dataset_ami))


52
898


In [126]:
test_dataset_icsi.head(5)

,meeting,original,extractive,abstractive
0,Bed004.A,"Hey , you 're not supposed to be drinking in h...","summarize: So , what I did for this this is uh...","It is not a working net yet, but identifying c..."
1,Bed004.A,"um , so we could sort of isolate them or whate...",summarize: So basically all I did was I took t...,"It is not a working net yet, but identifying c..."
2,Bed004.A,"Um , the Parse would be what verb they chose ,...","summarize: Um , the Parse would be what verb t...","It is not a working net yet, but identifying c..."
3,Bed004.B,"Mm - hmm . So , are are you trying to record ...","summarize: S so if you just number them "" one ...","The group decided to hire the ""wizard"" and con..."
4,Bed004.B,"So , so if you if we made if we wanted to make...","summarize: is , if we just do this , we could ...",There are potential problems from a combinator...


In [128]:
MAX_LEN = 512
SUMMARY_LEN= 150

# note here only uses the t5-small model.
tokenizer = T5Tokenizer.from_pretrained("t5-small")
test_set_icsi  = CustomDataset(test_dataset_icsi, tokenizer, MAX_LEN, SUMMARY_LEN)
test_set_ami  = CustomDataset(test_dataset_ami, tokenizer, MAX_LEN, SUMMARY_LEN)


In [129]:
# https://deeplizard.com/learn/video/kWVgvsejXsE#:~:text=The%20num_workers%20attribute%20tells%20the,sequentially%20inside%20the%20main%20process
# num_workers to default 0
# This means that the training process will work sequentially inside the main process. 
# After a batch is used during the training process and another one is needed, we read the batch data from disk.

TEST_BATCH_SIZE = 1 

test_params = {
  'batch_size': TEST_BATCH_SIZE,
  'shuffle': False,
  'num_workers': 0
  }
test_loader_icsi = DataLoader(test_set_icsi, **test_params)
test_loader_ami = DataLoader(test_set_ami, **test_params)

In [130]:
# https://towardsdatascience.com/fine-tuning-a-t5-transformer-for-any-summarization-task-82334c64c81

def test(epoch, tokenizer, model, device, loader, beams):
  #https://stackoverflow.com/questions/60018578/what-does-model-eval-do-in-pytorch
  model.eval()
  predictions = []
  actuals = []
  #rouge_metric = load_metric('rouge') 
  # https://datascience.stackexchange.com/questions/32651/what-is-the-use-of-torch-no-grad-in-pytorch
  with torch.no_grad():

    for _, data in enumerate(loader, 0):

      y = data['target_ids'].to(device, dtype = torch.long)
      ids = data['source_ids'].to(device, dtype = torch.long)
      mask = data['source_mask'].to(device, dtype = torch.long)

      generated_ids = model.generate(
          input_ids = ids,
          attention_mask = mask, 
          max_length=150, 
          num_beams=beams,
          repetition_penalty=2.5, 
          length_penalty=1.0, 
          early_stopping=True
          )
      preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
      target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
      if _%100==0:
          print(f'Completed {_}')
      predictions.extend(preds)
      actuals.extend(target)
  return predictions, actuals

In [131]:
AMI_PATH = "/content/drive/My Drive/W266/data/gold_abstractive_summary/goldsummary_AMI_as_test.csv"
ICSI_PATH = "/content/drive/My Drive/W266/data/gold_abstractive_summary/goldsummary_ICSI_as_test.csv"

amigold = pd.read_csv(AMI_PATH)
icsigold = pd.read_csv(ICSI_PATH)

def rouge_per_document(final_df,gold, test_dataset):
  merged_df = pd.concat([test_dataset.meeting, final_df.Generated_Abstractive_Summary], axis=1)
  merged_df["meetinglevel"] = merged_df.meeting.apply(lambda x: x.split(".")[0]) 

  gas_list =[]
  meeting_list = []
  generated_abstractive = ""
  for me in set(merged_df.meetinglevel):
    for gas in merged_df[merged_df.meetinglevel == me]['Generated_Abstractive_Summary']:
      generated_abstractive+= gas + " "
    gas_list.append(generated_abstractive)
    meeting_list.append(me)
    generated_abstractive = " "
  per_doc_summary = pd.DataFrame(
    {'Meeting': meeting_list,
     'Generated_Abstractive_Summary': gas_list
    })
  
  new_df = pd.merge(per_doc_summary, gold,  how='left', left_on='Meeting', right_on ='meeting')
  rouge_results_perdoc = compute_rouge_scores(new_df.Generated_Abstractive_Summary,
                                      new_df.abstractive)
  return rouge_results_perdoc
  

In [132]:
## ================= TESTING =================== ##
def testingprocess (modelsize, LEARNING_RATE,CP_TEMP_NAME, MODEL_NAME, folder,beams,gold,test_set,test_loader):
  print("strat testing")
  model = T5ForConditionalGeneration.from_pretrained(modelsize)
  model = model.to(device)
  optimizer = torch.optim.Adam(params = model.parameters(), lr=LEARNING_RATE)

  CP_PATH = "/content/drive/My Drive/W266/checkpoints/"+folder+ "/" + CP_TEMP_NAME +".pt"
  checkpoint = torch.load(CP_PATH)
  model.load_state_dict(checkpoint['model_state_dict'])
  optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
  print("strat epoch")
  predictions, actuals = test(1, tokenizer, model, device, test_loader,beams)
  final_df = pd.DataFrame({'Generated_Abstractive_Summary':predictions,
                            'Golden_Abstractive_Text':actuals})
  final_df.to_csv('/content/drive/My Drive/W266/final/'+MODEL_NAME +'_prediction.csv')

  rouge_results = compute_rouge_scores(final_df.Generated_Abstractive_Summary,
                                        final_df.Golden_Abstractive_Text)
  print("done row level Rouge")
  test_time = time.time() - start_test_time

  # amigold = pd.read_csv(AMI_PATH)
  # icsigold = pd.read_csv(ICSI_PATH)

  rouge_results_perdoc = rouge_per_document(final_df,gold,test_set)
  print("done document level Rouge")

  results = pd.DataFrame({
            'model': MODEL_NAME,
            'rouge1': rouge_results.get("rouge_1_f_score"), 
            'rougeL': rouge_results.get("rouge_l_f_score"),  
            'rouge2': rouge_results.get("rouge_2_f_score"),
            'rouge1_doclevel': rouge_results_perdoc.get("rouge_1_f_score"), 
            'rougeL_doclevel': rouge_results_perdoc.get("rouge_l_f_score"),  
            'rouge2_doclevel': rouge_results_perdoc.get("rouge_2_f_score"),
            'testTime': test_time
            },index=[0])
  results.to_csv('/content/drive/My Drive/W266/final/'+MODEL_NAME +'_rouge.csv')
  return rouge_results_perdoc,rouge_results

In [133]:
start_test_time = time.time()
LEARNING_RATE = 0.0005
MODEL_NAME = "ICSI_512_9"
CP_TEMP_NAME = 'epoch9' 
folder = "MSFT_50EPOCH_Intransit_ICSI512_NoNA"
beams = 9
gold = icsigold
test_set = test_dataset_icsi
testingprocess ("t5-small",LEARNING_RATE,CP_TEMP_NAME, 
                                  MODEL_NAME, folder,beams,gold,test_set,test_loader_icsi)


strat testing
strat epoch


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Completed 0
done row level Rouge
done document level Rouge


({'rouge_1_f_score': 0.24691853592978777,
  'rouge_2_f_score': 0.038219265353075904,
  'rouge_l_f_score': 0.20595295115555093},
 {'rouge_1_f_score': 0.15650915732937948,
  'rouge_2_f_score': 0.014400942429079331,
  'rouge_l_f_score': 0.11335229096312885})

In [134]:
start_test_time = time.time()
LEARNING_RATE = 0.0005
MODEL_NAME = "AMI_512_15"
CP_TEMP_NAME = 'epoch15' 
folder = "MSFT_50EPOCH_Intransit_AMI512_NoNA"
beams = 12
gold = amigold
test_set = test_dataset_ami
test_loader = test_loader_ami
testingprocess ("t5-small",LEARNING_RATE,CP_TEMP_NAME, 
                                  MODEL_NAME, folder,beams,gold,test_set,test_loader)



strat testing
strat epoch


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Completed 0
Completed 100
Completed 200
Completed 300
Completed 400
Completed 500
Completed 600
Completed 700
Completed 800
done row level Rouge
done document level Rouge


({'rouge_1_f_score': 0.2407445478960022,
  'rouge_2_f_score': 0.06733138141661116,
  'rouge_l_f_score': 0.26106556609284315},
 {'rouge_1_f_score': 0.15896372952509064,
  'rouge_2_f_score': 0.022142911276734817,
  'rouge_l_f_score': 0.13194669919855792})

In [119]:
## ================= TESTING =================== ##
def testingprocess_BERT (modelsize, LEARNING_RATE,CP_TEMP_NAME, MODEL_NAME, folder,beams,gold,test_set,test_loader):
  print("strat testing")
  model = T5ForConditionalGeneration.from_pretrained(modelsize)
  model = model.to(device)
  optimizer = torch.optim.Adam(params = model.parameters(), lr=LEARNING_RATE)

  CP_PATH = "/content/drive/My Drive/W266/checkpoints/"+folder+ "/" + CP_TEMP_NAME +".pt"
  checkpoint = torch.load(CP_PATH)
  model.load_state_dict(checkpoint['model_state_dict'])
  optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
  print("strat epoch")
  predictions, actuals = test(1, tokenizer, model, device, test_loader,beams)
  final_df = pd.DataFrame({'Generated_Abstractive_Summary':predictions,
                            'Golden_Abstractive_Text':actuals})
  final_df.to_csv('/content/drive/My Drive/W266/final/'+MODEL_NAME +'_prediction.csv')

  test_time = time.time() - start_test_time

  # amigold = pd.read_csv(AMI_PATH)
  # icsigold = pd.read_csv(ICSI_PATH)

  rouge_results_perdoc = rouge_per_document(final_df,gold,test_set)
  print("done document level Rouge")

  results = pd.DataFrame({
            'model': MODEL_NAME,
            'rouge1_doclevel': rouge_results_perdoc.get("rouge_1_f_score"), 
            'rougeL_doclevel': rouge_results_perdoc.get("rouge_l_f_score"),  
            'rouge2_doclevel': rouge_results_perdoc.get("rouge_2_f_score"),
            'testTime': test_time
            },index=[0])
  results.to_csv('/content/drive/My Drive/W266/final/'+MODEL_NAME +'_rouge.csv')
  return rouge_results_perdoc

In [123]:
start_test_time = time.time()
LEARNING_RATE = 0.001
MODEL_NAME = "ICSI_ex_largelr_34"
CP_TEMP_NAME = 'epoch34' 
folder = "50EPOCH_Intransit_ICSI1024_eax_largelr"
beams = 9
gold = icsigold
test_set = test_dataset_icsi
testingprocess_BERT("t5-small",LEARNING_RATE,CP_TEMP_NAME, 
                                  MODEL_NAME, folder,beams,gold,test_set,test_loader_icsi)


strat testing
strat epoch


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Completed 0
done document level Rouge


{'rouge_1_f_score': 0.24506177151011407,
 'rouge_2_f_score': 0.019061940082116213,
 'rouge_l_f_score': 0.19795215860618973}

In [124]:
start_test_time = time.time()
LEARNING_RATE = 0.001
MODEL_NAME = "AMI_ex_largelr_15"
CP_TEMP_NAME = 'epoch15' 
folder = "50EPOCH_Intransit_AMI1024_eax_largelr"
beams = 12
gold = amigold
test_set = test_dataset_ami
test_loader = test_loader_ami
testingprocess_BERT("t5-small",LEARNING_RATE,CP_TEMP_NAME, 
                                  MODEL_NAME, folder,beams,gold,test_set,test_loader)



strat testing
strat epoch


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Completed 0
done document level Rouge


{'rouge_1_f_score': 0.31107486599755413,
 'rouge_2_f_score': 0.07397523049180393,
 'rouge_l_f_score': 0.24337593412034625}

In [107]:
final_df= pd.read_csv('/content/drive/My Drive/W266/final/'+MODEL_NAME +'_prediction.csv')

In [108]:
merged_df = pd.concat([test_dataset_icsi.meeting, final_df.Generated_Abstractive_Summary], axis=1)
merged_df["meetinglevel"] = merged_df.meeting.apply(lambda x: x.split(".")[0]) 


In [112]:
merged_df.iloc[0]['Generated_Abstractive_Summary']

'results, if those prove appropriate, and how much responsibility rests on participants being available and checking their e-mail regularly. The group suggest sending reminder e-mails, although since many participants are local they can be contacted by other means if necessary. Checking of the NSA meetings has revealed that this non-native English meeting data contains transcription inaccuracies due to the use of foreign language terms and technical vocabulary. Additional topics covered more briefly in this meeting are disk space, the DARPA annual report,'

In [113]:

gas_list =[]
meeting_list = []
generated_abstractive = ""
for me in set(merged_df.meetinglevel):
  for gas in merged_df[merged_df.meetinglevel == me]['Generated_Abstractive_Summary']:
    generated_abstractive+= gas + " "
  gas_list.append(generated_abstractive)
  meeting_list.append(me)
  generated_abstractive = " "
per_doc_summary = pd.DataFrame(
  {'Meeting': meeting_list,
    'Generated_Abstractive_Summary': gas_list
  })

new_df = pd.merge(per_doc_summary, gold,  how='left', left_on='Meeting', right_on ='meeting')
print(new_df)

rouge_results_perdoc = compute_rouge_scores(new_df.Generated_Abstractive_Summary,
                                    new_df.abstractive)


  Meeting  ...                                        abstractive
0  Bed016  ...  The meeting was taken up by discussion about a...
1  Bmr005  ...  Topics discussed by the Berkeley Meeting Recor...
2  Bro018  ...  The ICSI Meeting Recorder Group met once more ...
3  Bed009  ...  The Berkeley Even Deeper Understanding group d...
4  Bmr019  ...  The Berkeley Meeting Recorder group discussed ...
5  Bed004  ...  A test run of the data collection design was v...

[6 rows x 4 columns]


In [115]:
new_df.iloc[0]['abstractive']

'The meeting was taken up by discussion about a thesis proposal and a talk about to take place at EML. The latter will present the work that is currently being done at ICSI including examples of inference of user intentions and of the recordings of the on-going data collection. The talk will also outline the theoretical (X-schemas, image schemas, Bayes-nets) and neural background. The thesis proposal, on the other hand, presents the idea of "construal" and makes claims as to how inferences are drawn in a probabilistic relational model by using information from the ontology, situation, user and discourse models. It was advised that more emphasis should be put on the role of construal in the understanding of metaphor and metonymy. Base constructions deal with the norm, while further general domain mechanisms determine how the constructions are invoked depending on the context. Several potential examples of polysemy were discussed in detail: "walk/run into", "on the bus", "out of film", "

In [116]:
new_df.iloc[0]['Generated_Abstractive_Summary']

"discussions were also made with regard to a visitor from research partner OGI These included making plans for upcoming experiments, clarifying definitions, and approaches which may or may not be against the rules of the Aurora project, alongside alternatives that would not be. Although the members of ICSI's Meeting Recorder Group at Berkeley had little progress to report, there were still a number of issues relating to their work to discuss. These included making plans for upcoming experiments, clarifying definitions, and approaches which may or may not be against the this includes reports on the progress of the groups main digit recogniser project, with interest on voice-activity detectors and voiced/unvoiced detection, work on acoustic feature detection, and research into dealing with reverberation. The group note that the annual report needs to be worked on for next week, and it is also suggested to hold recognition meetings separately, however these issues will be discussed in mor